In [131]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np

from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.metrics import accuracy_score, f1_score, make_scorer, recall_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, roc_curve, recall_score, accuracy_score, precision_score, plot_confusion_matrix, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


In [132]:
df = pd.read_csv('C:/Users/jordi/Documents/GitHub/SupplyChainResilience-Datathon-2022-UPC/JOINED_DATA/df_joined.csv')

In [133]:
df=df.drop(['product_id','order_id'],axis=1)
df = df.dropna()

In [134]:
df = pd.get_dummies(df, columns=['origin_port', 'X3pl', 'customs_procedures', 'logistic_hub', 'customer', 'material_handling'])

In [135]:
df['late_order']=df['late_order'].astype(int)

In [136]:
df['late_order_t'] = df['late_order']
df=df.drop(['late_order'],axis=1)

In [137]:
training_data, testing_data = train_test_split(df, test_size=0.05, random_state=25)

In [139]:
training_data=training_data.sample(frac=1)
train_nolate=training_data[training_data['late_order_t']==0]
train_late=training_data[training_data['late_order_t']==1]
train_nolate=train_nolate[:training_data['late_order_t'].value_counts()[1]]
print(train_nolate.shape)
print(train_late.shape)
training_data = pd.concat([train_nolate, train_late])
training_data = training_data.sample(frac=1)

(25787, 60)
(25787, 60)


In [140]:
def classificacio(clf, tuning=False):
  clf.fit(training_data.iloc[:,:-1], training_data.iloc[:,-1])
  preds = pd.Series(clf.predict(testing_data.iloc[:,:-1]), name='preds')
  reals = testing_data.iloc[:,-1]
  reals.name = 'reals'
  reals.index = range(len(preds))
  print('acc: ', accuracy_score(reals, preds))
  print('rec: ', recall_score(reals,preds, pos_label=1))
  print('prec: ',recall_score(reals,preds, pos_label=1))
  print('f1:', f1_score(reals, preds, pos_label=1))
  if tuning:
    print('best pars:', clf.best_params_)
    pd.crosstab(reals,preds)
  return [reals,preds]


In [93]:
resultats1={}

In [141]:
resultats1['RandomForestClassifier']=classificacio(RandomForestClassifier())
pd.crosstab(resultats1['RandomForestClassifier'][0],resultats1['RandomForestClassifier'][1])

acc:  0.7035739313244569
rec:  0.7163173652694611
prec:  0.7163173652694611
f1: 0.5307820299500832


preds,0,1
reals,,
0,3059,1313
1,379,957
